In [ ]:
import pickle
import pandas as pd

# Carregar o modelo 
with open('../modelo/modelo_treinado.pkl', 'rb') as f:
    modelo = pickle.load(f)

# Colunas usadas durante o treinamento
colunas_modelo = ['Formacao_Academica', 'Ano_Admissao', 'Nivel_Salarial', 
                  'Idade', 'Genero', 'Ficou_Sem_Tarefas', 'Experiencia_Area']

In [ ]:
def prever_deixou_empresa(Formacao, Ano_Admissao, Salario, Idade, Genero, Bench, Experiencia):
    # Criar um DataFrame com os inputs

    # Dicionários de mapeamento (categórico para numérico pois o modelo só 'enxerga' numeros)
    map_formacao = {
        'Bacharelado': 0,
        'Mestrado': 1,
        'Doutorado': 2
    }
    
    map_genero = {
        'Masculino': 0,
        'Feminino': 1,
    }
    
    map_bench = {
        'Sim': 1,
        'Não': 0
    }

    

    # Criar um DataFrame com os inputs (convertendo categóricos para numéricos)
    dados = pd.DataFrame({
        'Formacao_Academica': [map_formacao[Formacao]],
        'Ano_Admissao': [Ano_Admissao],
        'Nivel_Salarial': [Salario],
        'Idade': [Idade],
        'Genero': [map_genero[Genero]],
        'Ficou_Sem_Tarefas': [map_bench[Bench]],
        'Experiencia_Area': [Experiencia]
    })


    
    # Fazer a predição
    predicao = modelo.predict(dados)[0]
    return "Deixará a empresa" if predicao == 1 else "Permanecerá"

In [ ]:
import gradio as gr  # Importa a biblioteca Gradio para criar interfaces gráficas simples em Python

# Definir opções para campos categóricos
opcoes_formacao = ["Bacharelado", "Mestrado", "Doutorado"]  # Opções de formação acadêmica
opcoes_genero = ["Masculino", "Feminino"]                   # Opções de gênero
opcoes_bench = ["Sim", "Não"]                               # Se o funcionário já ficou sem tarefas ("on bench")

# Criar a interface com o Gradio usando blocos
with gr.Blocks() as app:
    gr.Markdown("Predição de Rotatividade de Funcionários")  # Título da interface

    # Primeira linha com dropdown e campo numérico
    with gr.Row():
        Formacao = gr.Dropdown(opcoes_formacao, label="Formação Acadêmica")  # Seleção de formação
        Ano_Admissao = gr.Number(label="Ano de Admissão", value=2020)        # Campo numérico com ano padrão

    # Segunda linha com slider, número e dropdown
    with gr.Row():
        Salario = gr.Slider(1, 3, step=1, label="Nível Salarial (1-3)")      # Slider para o nível salarial
        Idade = gr.Number(label="Idade", value=30)                           # Campo para idade
        Genero = gr.Dropdown(opcoes_genero, label="Gênero")                 # Seleção de gênero

    # Terceira linha com radio button e número
    with gr.Row():
        Bench = gr.Radio(opcoes_bench, label="Já ficou sem tarefas?")       # Pergunta binária (Sim/Não)
        Experiencia = gr.Number(label="Experiência na Área (anos)", value=5) # Anos de experiência

    # Botão de previsão
    btn = gr.Button("Prever")

    # Campo de saída (label)
    output = gr.Label(label="Resultado")

    # Define o que acontece ao clicar no botão:
    # chama a função 'prever_deixou_empresa' passando os inputs e exibindo a saída no label
    btn.click(
        fn=prever_deixou_empresa,  # Função de predição que precisa estar definida no código
        inputs=[Formacao, Ano_Admissao,  Salario, Idade, Genero, Bench, Experiencia],
        outputs=output
    )

# Inicia a aplicação com um link compartilhável
app.launch(share=True)